# Creating Raster Information Product using Raster Analytics

## Raster Analytics

ArcGIS Enterprise at 10.5 provides you with the ability to perform large raster analysis using distributed computing. This capability is provided in the `arcgis.raster.analytics` module and includes functionality to summarize data, analyze patterns, images, terrain and manage data. This sample show the capabilities of imagery layers and raster analytics.

## Imagery layers

In [ ]:
import arcgis
from arcgis.gis import GIS
from IPython.display import display

gis = GIS()

Here we're searching for multispectral landsat imagery layer:

In [ ]:
items = gis.content.search("Landsat 8 Views", "Imagery Layer", max_items=8)

In [ ]:
for item in items:
    display(item)

> NOTE: The index position in the `items` list for Landsat 8 Views may be different than the value below:

In [ ]:
landsat_item = items[4]

In [ ]:
imglyr = landsat_item.layers[0]

The code below cycles through and lists the Raster Functions published with the `imglyr`:

In [ ]:
for fn in imglyr.properties['rasterFunctionInfos']:
    print (fn['name'])

Let us create a map widget and load this layer

In [ ]:
marthasbasin = arcgis.geocoding.geocode("Marthas Basin, Montana")[0]

In [ ]:
map1 = gis.map(marthasbasin, zoomlevel=12)

In [ ]:
map1

![raster_function_cycle](r"../../static/imgs/04_sample_rasterinfo2.gif")

In [ ]:
map1.add_layer(imglyr)

The utility of raster functions is better seen when we interactively cycle through these raster functions and apply them to the map. The code below cycles through the first 6 raster functions stored with the Imagery Layer and a small time delay to illustrate. The image processing occurs on-the-fly at display resolution to show how the layer can be visualized using these different raster functions published with the layer.

In [ ]:
import time
from arcgis.raster.functions import apply

for fn in imglyr.properties['rasterFunctionInfos'][:6]:
    print(fn['name'])
    map1.remove_layers()
    map1.add_layer(apply(imglyr, fn['name']))
    time.sleep(4)  

## Raster functions

Developers can create their own **raster functions**, by chaining different raster functions. For instance, the code below is doing an Extract Band and extracting out the [4,5,3] band combination, and applying a Stretch to get the land-water boundary visualization that makes it easy to see where land is and where water is. Its worth noting that the raster function is applied at display resolution and only for the visible extent using on the fly image processing.

In [ ]:
from arcgis.raster.functions import stretch, extract_band

def process_bands(layer, bands):
    return stretch(extract_band(layer, bands), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, gamma=[1, 1, 1], dra=True)

Let us apply this raster function to the image layer to visualize the results.

In [ ]:
map2 = gis.map(marthasbasin, zoomlevel=12)
map2

![Marthas_Basin_bands](../../static/imgs/04_sample_rasterinfo3.png)

In [ ]:
map2.add_layer(process_bands(imglyr, [4, 5, 3]))

# Creating a Raster Information Product using Landsat 8 imagery

This part of the notebook shows how **Raster Analytics** (in ArcGIS Enterprise 10.5) can be used to generate a raster information product, by applying the same raster function across the extent of an image service on the portal. The raster function is applied at source resolution and creates an Information Product, that can be used for further analysis and visualization.

In [ ]:
portal = GIS("https://pythonapi.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

In [ ]:
montana_landsat = portal.content.search("ImgSrv_Landsat_Montana2015")[0]
montana_landsat

In the code below, we use extract and stretch the [7, 5, 2] band combination. This improves visibility of fire and burn scars by pushing further into the SWIR range of the electromagnetic spectrum, as there is less susceptibility to smoke and haze generated by a burning fire.

In [ ]:
montana_lyr = montana_landsat.layers[0]

In [ ]:
fire_viz = process_bands(montana_lyr, [7, 5, 2])
fire_viz

![fire_visualization_MT](../../static/imgs/04_sample_rasterinfo4.png)

We can use the `save` method to apply the raster function across the entire extext of the input image layer at source resolution, and presist the result in an output image layer. This creates a raster product similar that can be used for further analysis and visualization.

In [ ]:
arcgis.env.verbose = True

In [ ]:
montana_fires_lyr = fire_viz.save('Montana_Burn_scars')

In [ ]:
montana_fires_lyr

In [ ]:
base_map = portal.map(marthasbasin, 12)

natural_color_map = portal.map(marthasbasin, 12)
natural_color_map.add_layer(montana_landsat)

false_color_map = portal.map(marthasbasin, 12)
false_color_map.add_layer(montana_fires_lyr)

We can compare the natural color and false color images uaing a tabbed widget. 

In the false color image the red and brownish pixels correspond to burn scars of the fire:

In [ ]:
import ipywidgets as widgets

tab = widgets.Tab([base_map, natural_color_map, false_color_map])
tab.set_title(0, 'Basemap')
tab.set_title(1, 'Natural Color')
tab.set_title(2, 'False Color')
tab

![tabbed_widget](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_raster_analysis_01.gif)

Thus using the same raster function, we were able to both visualize on the fly and derive a persisted image service using distributed raster analysis.